In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image, ImageDraw
from quickdraw import QuickDrawDataGroup
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


In [ ]:
CATEGORIES = ["cat", "dog", "tree", "house", "car", "apple", "chair", "bird", "fish", "flower"]
NUM_CLASSES = len(CATEGORIES)
IMG_SIZE = (28, 28)
IMAGES_PER_CATEGORY = 1000

In [ ]:
def load_and_preprocess_data():
    x_data, y_data = [], []
    for idx, category in enumerate(CATEGORIES):
        qd_group = QuickDrawDataGroup(category, max_drawings=IMAGES_PER_CATEGORY, recognized=True)
        images = []
        for drawing in qd_group.drawings:
            img = drawing.get_image(stroke_width=3).resize(IMG_SIZE).convert("L")
            img_array = 255 - np.array(img)
            images.append(img_array)
        x_data.extend(images)
        y_data.extend([idx] * len(images))

    x_data = np.array(x_data).reshape(-1, 28, 28, 1).astype('float32') / 255
    y_data = tf.keras.utils.to_categorical(y_data, NUM_CLASSES)
    return train_test_split(x_data, y_data, test_size=0.2, random_state=42)


In [ ]:
def build_and_train_model():
    x_train, x_test, y_train, y_test = load_and_preprocess_data()
    
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', name='feature_layer')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))
    
    from sklearn.decomposition import PCA

    feature_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer('feature_layer').output)
    x_features = feature_model.predict(x_test)
    pca = PCA(n_components=2)
    x_tsne = pca.fit_transform(x_features)

    return model, feature_model, x_test, y_test, pca, x_tsne



In [ ]:
class ObjectRecognizerApp:
    def __init__(self, root, model, feature_model, x_test, y_test, tsne, x_tsne):
        self.root = root
        self.model = model
        self.feature_model = feature_model
        self.x_test = x_test
        self.y_test = y_test
        self.tsne = tsne
        self.x_tsne = x_tsne
        
        self.root.title("Object Recognizer")
        self.canvas = tk.Canvas(root, width=200, height=200, bg='white')
        self.canvas.pack()
        
        self.predict_btn = tk.Button(root, text="Predict", command=self.predict_object)
        self.predict_btn.pack()
        self.clear_btn = tk.Button(root, text="Clear", command=self.clear_canvas)
        self.clear_btn.pack()
        self.doodle_btn = tk.Button(root, text="Open Doodle Dialog", command=self.open_doodle_dialog)
        self.doodle_btn.pack()
        self.result_label = tk.Label(root, text="Draw an object and click Predict")
        self.result_label.pack()

        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)
        self.last_x, self.last_y = None, None

        self.canvas.bind("<B1-Motion>", self.draw_line)
        self.canvas.bind("<ButtonRelease-1>", self.reset_last)

    def draw_line(self, event):
        if self.last_x is not None and self.last_y is not None:
            self.canvas.create_line(self.last_x, self.last_y, event.x, event.y, width=10, fill='black')
            self.draw.line([self.last_x, self.last_y, event.x, event.y], fill=0, width=10)
        self.last_x, self.last_y = event.x, event.y

    def reset_last(self, event):
        self.last_x, self.last_y = None, None

    def clear_canvas(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)
        self.result_label.config(text="Draw an object and click Predict")

    def preprocess_image(self, image):
        img = image.resize((28, 28))
        img_array = 255 - np.array(img)
        img_array = img_array.astype('float32') / 255
        return img_array.reshape(1, 28, 28, 1)

    def predict_object(self):
        img_array = self.preprocess_image(self.image)
        prediction = self.model.predict(img_array)
        object_idx = np.argmax(prediction)
        object_name = CATEGORIES[object_idx]
        confidence = np.max(prediction)
        self.result_label.config(text=f"Predicted: {object_name} ({confidence:.2%})")

    def open_doodle_dialog(self):
        dialog = tk.Toplevel(self.root)
        dialog.title("Doodle and Visualize")
        canvas = tk.Canvas(dialog, width=200, height=200, bg='white')
        canvas.pack()

        image = Image.new("L", (200, 200), 255)
        draw = ImageDraw.Draw(image)
        last = {"x": None, "y": None}

        def draw_line(event):
            if last["x"] is not None and last["y"] is not None:
                canvas.create_line(last["x"], last["y"], event.x, event.y, width=10, fill='black')
                draw.line([last["x"], last["y"], event.x, event.y], fill=0, width=10)
            last["x"], last["y"] = event.x, event.y

        def reset_last(event):
            last["x"], last["y"] = None, None

        canvas.bind("<B1-Motion>", draw_line)
        canvas.bind("<ButtonRelease-1>", reset_last)

        result_label = tk.Label(dialog, text="Draw and click Predict")
        result_label.pack()

        def predict_and_visualize():
            img_array = self.preprocess_image(image)
            prediction = self.model.predict(img_array)
            object_idx = np.argmax(prediction)
            object_name = CATEGORIES[object_idx]
            confidence = np.max(prediction)
            result_label.config(text=f"Predicted: {object_name} ({confidence:.2%})")

            doodle_features = self.feature_model.predict(img_array)
            self.visualize_doodle(doodle_features, object_name)

        predict_btn = tk.Button(dialog, text="Predict and Visualize", command=predict_and_visualize)
        predict_btn.pack()

        def clear_doodle():
            canvas.delete("all")
            image.paste(255, (0, 0, 200, 200))
            result_label.config(text="Draw and click Predict")
            last["x"], last["y"] = None, None

        clear_btn = tk.Button(dialog, text="Clear", command=clear_doodle)
        clear_btn.pack()

import matplotlib.cm as cm
from matplotlib.colors import to_hex

from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon
from matplotlib import cm
from matplotlib.colors import to_hex

def visualize_doodle(self, doodle_features, doodle_label):
    all_features = np.vstack([self.feature_model.predict(self.x_test), doodle_features])
    all_tsne = self.pca.fit_transform(all_features)

    fig, ax = plt.subplots(figsize=(12, 10))
    cmap = cm.get_cmap('tab10', len(CATEGORIES))
    colors = [to_hex(cmap(i)) for i in range(len(CATEGORIES))]
    scatter_handles = []

    for i, category in enumerate(CATEGORIES):
        mask = np.argmax(self.y_test, axis=1) == i
        pts = all_tsne[mask]
        color = colors[i]
        
        # Scatter points
        handle = ax.scatter(
            pts[:, 0], pts[:, 1],
            label=category,
            s=50,
            alpha=0.5,
            edgecolors='w',
            linewidths=0.5,
            color=color
        )
        scatter_handles.append(handle)
        
        # Optional: draw convex hull
        if len(pts) >= 3:
            hull = ConvexHull(pts)
            polygon = Polygon(pts[hull.vertices], closed=True, edgecolor=color,
                              facecolor=color, alpha=0.08, linewidth=1.5)
            ax.add_patch(polygon)
        
        # Label cluster center
        center = np.mean(pts, axis=0)
        ax.text(center[0], center[1], category, fontsize=10, weight='bold',
                ha='center', va='center', color=color, alpha=0.8)

    # Plot the doodle
    doodle_point = all_tsne[-1]
    ax.scatter(
        doodle_point[0], doodle_point[1],
        marker='*',
        s=400,
        c='black',
        edgecolors='yellow',
        linewidths=2,
        zorder=10,
        label=f"Doodle: {doodle_label}"
    )
    ax.set_title("Visualizing Your Doodle in Feature Space", fontsize=18, pad=15)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.savefig("feature_space_doodle.png")
    plt.show()



In [ ]:
model, feature_model, x_test, y_test, pca, x_tsne = build_and_train_model()


root = tk.Tk()
app = ObjectRecognizerApp(root, model, feature_model, x_test, y_test, pca, x_tsne)
root.mainloop()


In [ ]:
model, feature_model, x_test, y_test, pca, x_tsne = build_and_train_model()


root = tk.Tk()
app = ObjectRecognizerApp(root, model, feature_model, x_test, y_test, pca, x_tsne)
root.mainloop()
